In [5]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark
import numpy as np

In [6]:
spark = SparkSession.builder.master("spark://master:7077").appName("preprocessForML").config("spark.executor.memory", "6gb").getOrCreate()
sc = spark.sparkContext

In [7]:
# RDD_4mer = sc.textFile("/kmer_counting/output_java_spark_4mer/part-00000")
# kmer_seed = 77
# embedding_kmers = np.array([str(sub.split(',')[0][1:]) for sub in RDD_4mer.take(3000)])
# embedding_kmers_idxs =  np.random.default_rng(seed=kmer_seed).choice(range(3000),size=1024,replace=False)
# embedding_kmers = list(embedding_kmers[embedding_kmers_idxs])

embedding_kmers = []

with open('best_kmers.txt','r') as f:
    for line in f:
        embedding_kmers.append(line.strip())
        
print(embedding_kmers)

['VAAL', 'AGAT', 'DALK', 'GAAL', 'GGAA', 'LAAT', 'LSPS', 'ASPS', 'LGLA', 'PLPP', 'EVEE', 'VLLP', 'SAAE', 'SRLL', 'TVVA', 'AGGR', 'ATVA', 'VAGV', 'AIVA', 'LAST', 'LQRL', 'SAAV', 'LEKV', 'DRLL', 'LLVG', 'ASAV', 'FAAL', 'SSTS', 'AAAG', 'LGVV', 'LAGL', 'ARIL', 'GVPV', 'AIAA', 'ALLL', 'EALE', 'LARV', 'AAVV', 'VAAS', 'ERLR', 'GLDG', 'AGVD', 'AAAN', 'EATA', 'VVVT', 'AVEA', 'GLGR', 'LGIV', 'RAVE', 'VLAL', 'VAGA', 'AERE', 'GASS', 'ERLG', 'TLAE', 'AALL', 'VGLL', 'TSSG', 'ERVL', 'ILSA', 'AGAS', 'AELS', 'EALK', 'LLES', 'AIAT', 'ASLL', 'LGAA', 'AAVR', 'AGAG', 'EAGV', 'IAAR', 'EELL', 'IVLL', 'LGAL', 'EAGR', 'ATSL', 'AVIL', 'LLDR', 'DRLA', 'LPAD', 'LAVP', 'VRDA', 'EAAR', 'AVVG', 'IAIA', 'ALSE', 'LSAL', 'LAVG', 'LALA', 'PAAT', 'LLSG', 'ATDA', 'KVLL', 'DLLK', 'FAGL', 'FLAA', 'LLDA', 'AAPG', 'NLAL', 'ADEA', 'ALDD', 'AALI', 'EAAL', 'RQLL', 'SVVV', 'TAAD', 'RVLA', 'RAVA', 'RLKE', 'ADAA', 'IDAA', 'LAVL', 'ELAK', 'GLLE', 'EALA', 'GGAP', 'GDGS', 'SAAT', 'TSSL', 'LSAT', 'GVRL', 'VALR', 'GSGK', 'EAGG', 'SAAA',

In [8]:
positive_idx = sc.textFile('/data/positiv_labels.txt').collect()
print(type(positive_idx))

sequences = sc.textFile('/data/swissprot.txt')

def encode(seq):
    bitlist = [1 if kmer in seq else 0 for kmer in embedding_kmers]
    out = []
    i=0
    for bit in bitlist:
        if i%32==0:
            out.append(0)
        out[-1] = (out[-1]<<1) | bit
        i+=1
    return tuple(out)

def create_tuple(x):
    out = []
    data = x.split()

    label = int (data[0] in positive_idx)
    out.append(label)

    bitlist = [1 if kmer in data[1] else 0 for kmer in embedding_kmers]
    i = 0
    for bit in bitlist:
        if i%32==0:
            out.append(0)
        out[-1] = (out[-1]<<1) | bit
        i+=1
    return tuple(out)

def create_tuple_unencode(x):
    out = []
    data = x.split()
    
    label = int (data[0] in positive_idx)
    out.append(label)
    
    out += [1 if kmer in data[1] else 0 for kmer in embedding_kmers]
    
    return tuple(out)

<class 'list'>


In [ ]:
'''
Encoded Version
'''

tuples = sequences.map(lambda x: create_tuple(x)).coalesce(1).saveAsTextFile('/MLInput')

In [ ]:
'''
Encoded Version
'''

''' move the output file 'part-00000' from hdfs to local '''
# hdfs dfs -copyToLocal /MLInput/part-00000 ~/jupyter_notebook /

# Write encoded version of sequences: for supervised & unsupervised learning
with open('part-00000', 'r') as f1:
    with open('MLInput.csv', 'w') as f2:
        columns = ['labels,', ','.join(['f'+str(i) for i in range(1,33)]), '\n']
        f2.writelines(columns)
        for line in f1:
            nums = line.strip()[1:-1].split(',')
            nums = [i.strip() for i in nums]
            f2.write(','.join(nums)+'\n')
            
with open('part-00000', 'r') as f1:
    with open('MLInput_cluster.csv', 'w') as f2:
        f2.write(','.join(['f'+str(i) for i in range(1,33)])+'\n')
        for line in f1:
            nums = line.strip()[3:-1].split(',')
            nums = [i.strip() for i in nums]
            f2.write(','.join(nums)+'\n')

''' move the processed file from local to hdfs '''
# hdfs dfs -copyFromLocal MLInput.csv /MLInput/
# hdfs dfs -copyFromLocal MLInput_cluster.csv /MLInput/

In [ ]:
'''
Unencoded Version
'''

tuples = sequences.map(lambda x: create_tuple_unencode(x)).coalesce(1).saveAsTextFile('/MLInput_u')

In [10]:
'''
Unencoded Version
'''

''' move the output file 'part-00000' from hdfs to local '''
# hdfs dfs -copyToLocal /MLInput_u/part-00000 ~/
# mv part-00000 part-00000_u
# mv part-00000_u ~/jupyter_notebook

# Write encoded version of sequences: for supervised & unsupervised learning
with open('part-00000_u', 'r') as f1:
    with open('MLInput_cluster_u.csv', 'w') as f2:
        f2.write(','.join(['f'+str(i) for i in range(1,1025)])+'\n')
        for line in f1:
            nums = line.strip()[3:-1].split(',')
            nums = [i.strip() for i in nums]
            f2.write(','.join(nums)+'\n')
        
            
''' move the processed file from local to hdfs '''
# hdfs dfs -copyFromLocal MLInput_u.csv /MLInput_u/
# hdfs dfs -copyFromLocal MLInput_cluster_u.csv /MLInput_u/

' move the processed file from local to hdfs '

In [11]:
with open('part-00000_u', 'r') as f1:
    with open('MLInput_u.csv', 'w') as f2:
        columns = ['labels,', ','.join(['f'+str(i) for i in range(1,1025)]), '\n']
        f2.writelines(columns)
        for line in f1:
            nums = line.strip()[1:-1].split(',')
            nums = [i.strip() for i in nums]
            f2.write(','.join(nums)+'\n')